In [2]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import concurrent.futures
from selenium.common.exceptions import StaleElementReferenceException


In [1]:
data = {
    "keywords": "jokowi",
    "since_time": "2023-10-18",
    "until_time": "2023-10-19"
}

In [4]:
def cek_totalPagination(keywords, since_time, until_time):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    since_time = datetime.strptime(since_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    until_time = datetime.strptime(until_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page=1"
    
    response = requests.get(url,headers=headers) 
    soup = BeautifulSoup(response.text, "html.parser")
    cek = soup.find('span', {"class": "fl text"})
    hasil_text = cek.get_text()

    # Use regular expression to extract the number
    result = re.search(r'\d+', hasil_text)
    if result:
        hasil_number = int(result.group(0))
        hasil_divided = hasil_number / 9  # Divide the number by 9
        rounded_result = round(hasil_divided)  # Round to the nearest whole number
        return rounded_result
    else:
        return 0  # Return 0 if no number was found


In [5]:
jumlah_index = cek_totalPagination(data["keywords"], data["since_time"], data["until_time"])
print(f"Total results: {jumlah_index}")

Total results: 20


In [30]:
def scrape_links(page_number,keywords, since_time, until_time):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    since_time = datetime.strptime(since_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    until_time = datetime.strptime(until_time, "%Y-%m-%d").strftime("%d/%m/%Y")
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={since_time}&todatex={until_time}&page={page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [31]:
# import concurrent.futures

# def scrape_links_with_threads(max_threads, jumlah_index, data):
#     links = []
#     with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
#         for page_number in range(1, jumlah_index + 1):
#             thread = executor.submit(scrape_links, page_number, data["keywords"], data["since_time"], data["until_time"])
#             links.extend(thread.result())

#     return links

# # Define the maximum number of concurrent threads you want to use
# max_threads = 8  # Adjust this number based on your system's capabilities

# # Call the function to scrape links
# scraped_links = scrape_links_with_threads(max_threads, jumlah_index, data)

# # Now you can use 'scraped_links' for further processing
# print("Total Links:", len(scraped_links))


In [ ]:
if __name__ == "__main__":
    # Create a ThreadPoolExecutor with a number of threads matching CPU cores
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit scraping tasks
        for page_number in range(1, jumlah_index + 1):
            executor.submit(scrape_links, page_number)

    # Print the total number of links
    print("Total Links:", len(links))

In [32]:
# def scrape_links_in_threads(data, jumlah_index):
#     links = []
#     threads_link = []
    
#     def scrape_links_for_page(page_number):
#         links.extend(scrape_links(page_number, data["keywords"], data["since_time"], data["until_time"]))

#     for page_number in range(1, jumlah_index + 1):
#         thread = threading.Thread(target=scrape_links_for_page, args=(page_number,))
#         thread.start()
#         threads_link.append(thread)

#     for thread in threads_link:
#         thread.join()
    
#     print("Total Links:", len(links))
#     return links

# # Call the function to scrape links in multiple threads
# scraped_links = scrape_links_in_threads(data, jumlah_index)

Scraped 9 links from page 3
Scraped 9 links from page 11
Scraped 9 links from page 20
Scraped 9 links from page 18
Scraped 9 links from page 19
Scraped 9 links from page 4
Scraped 9 links from page 33
Scraped 9 links from page 15
Scraped 9 links from page 30
Scraped 9 links from page 25
Scraped 9 links from page 32
Scraped 9 links from page 34
Scraped 9 links from page 16
Scraped 9 links from page 31
Scraped 9 links from page 22
Scraped 9 links from page 6
Scraped 9 links from page 23
Scraped 9 links from page 14
Scraped 9 links from page 5
Scraped 9 links from page 35
Scraped 9 links from page 24
Scraped 9 links from page 21
Scraped 9 links from page 29
Scraped 9 links from page 26
Scraped 9 links from page 17
Scraped 9 links from page 7
Scraped 9 links from page 13
Scraped 9 links from page 9
Scraped 9 links from page 10
Scraped 9 links from page 27
Scraped 9 links from page 12
Scraped 9 links from page 28
Scraped 9 links from page 8
Scraped 9 links from page 1
Scraped 9 links from p

In [33]:
print(scraped_links)

['https://www.detik.com/sulsel/berita/d-6990657/paripurna-hut-ke-354-sulsel-kejati-prihatin-angka-korupsi-capai-92-kasus', 'https://news.detik.com/pemilu/d-6990613/mahfud-pakai-kemeja-putih-yang-dijahit-untuk-pilpres-2019-bareng-jokowi', 'https://www.detik.com/sulsel/berita/d-6990597/pangdam-hasanuddin-soroti-inflasi-kekeringan-saat-paripurna-hut-sulsel', 'https://www.detik.com/jateng/berita/d-6990558/kritik-putusan-mk-bem-uns-posting-video-anwar-usman-gibran-dan-jokowi', 'https://www.detik.com/jateng/berita/d-6990555/gibran-tepis-rumor-ngurus-skck-di-polda-jateng', 'https://news.detik.com/pemilu/d-6990512/relawan-berkaus-golfud-golongan-mahfud-muncul-di-tugu-proklamasi', 'https://news.detik.com/berita/d-6990499/akting-yosep-pembunuh-istri-anak-tak-tanggung-tanggung-sempat-surati-jokowi', 'https://www.detik.com/sumut/berita/d-6990412/apakah-hari-santri-2023-libur-nasional-cek-infonya-di-sini', 'https://news.detik.com/berita/d-6990387/pendukung-di-indramayu-sambut-putusan-mk-doakan-gibr

In [34]:
# import concurrent.futures

# # Define the maximum number of concurrent threads you want to use
# max_threads = 8  # Adjust this number based on your system's capabilities

# links = []
# with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
#     for page_number in range(1, jumlah_index + 1):
#         thread = executor.submit(scrape_links, page_number, data["keywords"], data["since_time"], data["until_time"])
#         links.extend(thread.result())

# # No need for manual thread joining in this case

# print("Total Links:", len(links))


In [35]:
# import concurrent.futures

# # Define the maximum number of concurrent threads you want to use
# max_threads = 8  # Adjust this number based on your system's capabilities

# links = []
# with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
#     for page_number in range(1, jumlah_index + 1):
#         thread = executor.submit(scrape_links, page_number, data["keywords"], data["since_time"], data["until_time"])
#         links.extend(thread.result())

# # No need for manual thread joining in this case

# print("Total Links:", len(links))


In [36]:
# links = []
# threads_link = []
# for page_number in range(1, jumlah_index + 1):
#     thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,data["keywords"], data["since_time"], data["until_time"])))
#     thread.start()
#     threads_link.append(thread)

# for thread in threads_link:
#     thread.join()
# print("Total Links:", len(links))


In [37]:
# results = []

In [38]:
# def scrape_url(url,keywords):
#     try:
#         headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
#         }
#         response = requests.get(url,headers=headers)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, 'html.parser')
            
            
#             # Judul Berita
#             title_elem = soup.find('h1', {"class": "detail__title"})
#             if title_elem:
#                 title_text = title_elem.text.strip()
#             else:
#                 title_text = "Title not found"
#             # Author berita
#             author_elem = soup.find('div', {"class": "detail__author"})
#             if author_elem:
#                 author_text = author_elem.get_text()
#                 author_text = author_text.split('-')[0].strip()
#             else:
#                 author_text = "Author not found"     
#             # tanggal berita
#             date_elem = soup.find('div', {"class": "detail__date"})
#             if date_elem:
#                 date_text = date_elem.text.strip()
#             else:
#                 date_text = "Date not found"
#             #     # Category berita
#             category_elements = soup.find('div',{"class": "page__breadcrumb"})
#             if category_elements:
#                 category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
#                 category_text= category_text.text.strip()
#             else:
#                 category_text = "Category not found"
#             #     # Content Berita
#             body_elem = soup.find('div', {"class": "detail__body"})
            
#             if body_elem:
#                 content_elem = body_elem.find_all('p')
#                 content_text = ""
#                 for p in content_elem:
#                     content_text += p.text.strip() + "\n"
                
#                 if content_text.strip():
#                     content_text=content_text
#                 else:
#                     content_text ="Content not found"
#             else:
#               content_text ="Content not found"

#             nama_berita_match = re.search(r'https://(?:www\.)?([a-zA-Z0-9.-]+)\.com', url)
#             if nama_berita_match:
#                 nama_berita = nama_berita_match.group(1)
#             else:
#                 nama_berita = "Nama_berita not found"
#             results.append({'title': title_text,
#                             'keywords':keywords,
#                             'author' : author_text,
#                             'category':category_text,
#                             'date': date_text,
#                             'content' : content_text,
#                             'nama_berita' : nama_berita,
#                             'link' : url})
#             print(f"Done Get data  {url}")
#         else:
#             print(f"Failed to retrieve data from {url}")

#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching URL '{url}': {e}")
#     except Exception as e:
#         print(f"Error processing URL '{url}': {e}")

In [39]:
def scrape_url(url, keywords,max_retries=3):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Extract data from the web page
                    title_elem = soup.find('h1', {"class": "detail__title"})
                    title_text = title_elem.text.strip() if title_elem else "Title not found"

                    author_elem = soup.find('div', {"class": "detail__author"})
                    author_text = author_elem.get_text().split('-')[0].strip() if author_elem else "Author not found"

                    date_elem = soup.find('div', {"class": "detail__date"})
                    date_text = date_elem.text.strip() if date_elem else "Date not found"

                    category_elements = soup.find('div', {"class": "page__breadcrumb"})
                    category_text = category_elements.find('a', {"dtr-sec": "breadcrumbkanal"}).text.strip() if category_elements else "Category not found"

                    body_elem = soup.find('div', {"class": "detail__body"})
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = "\n".join(p.text.strip() for p in content_elem)
                        content_text = content_text.replace('\n', ' ')
                    else:
                        content_text = "Content not found"

                    nama_berita_match = re.search(r'https://(?:www\.)?([a-zA-Z0-9.-]+)\.com', url)
                    nama_berita = nama_berita_match.group(1) if nama_berita_match else "Nama_berita not found"

                    return {
                        'title': title_text,
                        'keywords': keywords,
                        'author': author_text,
                        'category': category_text,
                        'date': date_text,
                        'content': content_text,
                        'nama_berita': nama_berita,
                        'link': url
                    }
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [40]:
# def scrape_url(url, keywords):
#     try:
#         headers = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
#         }
#         response = requests.get(url, headers=headers)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, 'html.parser')

#             # Extract data from the web page
#             title_elem = soup.find('h1', {"class": "detail__title"})
#             title_text = title_elem.text.strip() if title_elem else "Title not found"

#             author_elem = soup.find('div', {"class": "detail__author"})
#             author_text = author_elem.get_text().split('-')[0].strip() if author_elem else "Author not found"

#             date_elem = soup.find('div', {"class": "detail__date"})
#             date_text = date_elem.text.strip() if date_elem else "Date not found"

#             category_elements = soup.find('div', {"class": "page__breadcrumb"})
#             category_text = category_elements.find('a', {"dtr-sec": "breadcrumbkanal"}).text.strip() if category_elements else "Category not found"

#             body_elem = soup.find('div', {"class": "detail__body"})
#             if body_elem:
#                 content_elem = body_elem.find_all('p')
#                 content_text = "\n".join(p.text.strip() for p in content_elem)
#             else:
#                 content_text = "Content not found"

#             nama_berita_match = re.search(r'https://(?:www\.)?([a-zA-Z0-9.-]+)\.com', url)
#             nama_berita = nama_berita_match.group(1) if nama_berita_match else "Nama_berita not found"

#             return {
#                 'title': title_text,
#                 'keywords': keywords,
#                 'author': author_text,
#                 'category': category_text,
#                 'date': date_text,
#                 'content': content_text,
#                 'nama_berita': nama_berita,
#                 'link': url
#             }
#         else:
#             print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching URL '{url}': {e}")
#     except Exception as e:
#         print(f"Error processing URL '{url}': {e}")

#     # Return None in case of an error or if data extraction fails
#     return None


In [41]:
import os

num_cores = os.cpu_count()

if num_cores is not None:
    print(f"Number of CPU cores available: {num_cores}")
else:
    print("Number of CPU cores could not be determined.")

Number of CPU cores available: 8


In [42]:
def scrape_urls_in_threads(scraped_links, data):
    results = []
    threads = []

    def scrape_url_and_store_result(url):
        result = scrape_url(url, data["keywords"])
        if result:
            results.append(result)

    for url in scraped_links:
        thread = threading.Thread(target=scrape_url_and_store_result, args=(url,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    return results

# Call the function to scrape URLs in multiple threads and store the results
results = scrape_urls_in_threads(scraped_links, data)

In [43]:
# import concurrent.futures

# def scrape_and_store_content_in_threads(scraped_links, max_threads, data):
#     results = []
#     with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
#         future_to_url = {executor.submit(scrape_url, url, data["keywords"]): url for url in scraped_links}
#         for future in concurrent.futures.as_completed(future_to_url):
#             url = future_to_url[future]
#             try:
#                 result = future.result()
#                 if result:
#                     results.append(result)
#                 else:
#                     print(f"Failed to scrape data from {url}")
#             except Exception as e:
#                 print(f"Error processing URL '{url}': {e}")

#     return results

# # Define the maximum number of concurrent threads you want to use
# max_threads = 8  # Adjust this number based on your system's capabilities

# # Call the function to scrape and store content in threads
# results = scrape_and_store_content_in_threads(scraped_links, max_threads, data)

# # Continue with any further processing after threads have completed


In [49]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df

hasil scrapping 315


,title,keywords,author,category,date,content,nama_berita,link
0,"Jokowi Restui Kenaikan Tukin Kemensos, Risma D...",jokowi,Herdi Alif Al Hikam,Berita Ekonomi Bisnis,"Kamis, 19 Okt 2023 10:05 WIB",Kabar baik bagi pegawai Kementerian Sosial. Pr...,finance.detik,https://finance.detik.com/berita-ekonomi-bisni...
1,"Gugatan PSI Ditolak MK, Yusril: Dugaan 'Mahkam...",jokowi,Dwi Rahmawati,Pemilu,"Senin, 16 Okt 2023 12:45 WIB",Ketua Umum (Ketum) PBB Yusril Ihza Mahendra me...,news.detik,https://news.detik.com/pemilu/d-6984690/gugata...
2,Akting Yosep Pembunuh Istri-Anak Tak Tanggung-...,jokowi,"Rifat Alhamidi, Sudirman Wamad",Berita,"Kamis, 19 Okt 2023 11:08 WIB",Yosep Hidayah sempat mengirim surat ke Preside...,news.detik,https://news.detik.com/berita/d-6990499/akting...
3,Gibran Maju Pilpres? Jokowi: Saya Tak Campuri ...,jokowi,Eva Safitri,Pemilu,"Senin, 16 Okt 2023 20:21 WIB",Presiden Joko Widodo (Jokowi) buka suara soal ...,news.detik,https://news.detik.com/pemilu/d-6985715/gibran...
4,"Puan Penasaran Arah Dukungan Jokowi, TPN: 100%...",jokowi,Isal Mawardi,Pemilu,"Kamis, 19 Okt 2023 08:18 WIB",Ketua DPP PDIP Puan Maharani bertanya-tanya te...,news.detik,https://news.detik.com/pemilu/d-6990184/puan-p...
...,...,...,...,...,...,...,...,...
310,Bandara Kertajati Bisa Dikelola Investor Asing,jokowi,Shafira Cendra Arini,Infrastruktur,"Rabu, 18 Okt 2023 16:45 WIB",Menteri Perhubungan Budi Karya Sumadi menyebut...,finance.detik,https://finance.detik.com/infrastruktur/d-6989...
311,"Dinilai Bisa Merapat ke Golkar Jalur AMPI, Gib...",jokowi,Tara Wahyu NV,Berita,"Kamis, 19 Okt 2023 14:12 WIB","Nama Wali Kota Solo, Gibran Rakabuming Raka di...",detik,https://www.detik.com/jateng/berita/d-6990979/...
312,"Situasi Gaza Memburuk, Menlu RI Tinggalkan Chi...",jokowi,Yulida Medistiara,Berita,"Rabu, 18 Okt 2023 16:22 WIB",Menteri Luar Negeri RI (Menlu) Retno Marsudi s...,news.detik,https://news.detik.com/berita/d-6989193/situas...
313,Perindo Pede Duet Ganjar-Mahfud Bisa Patahkan ...,jokowi,Helmy Akbar,Nusra,"Kamis, 19 Okt 2023 15:22 WIB",Dewan Pimpinan Wilayah (DPW) Perindo Provinsi ...,detik,https://www.detik.com/bali/nusra/d-6991185/per...


In [45]:
# import concurrent.futures

# # Define the maximum number of concurrent threads you want to use
# max_threads = 8  # Adjust this number based on your system's capabilities

# threads = []
# with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
#     for url in scraped_links:
#         thread = executor.submit(scrape_url, url, data["keywords"])
#         threads.append(thread)

# # Wait for all threads to complete
# for thread in threads:
#     thread.result()


In [46]:
# print(results)

In [47]:
# threads = []
# for url in links:
#     thread = threading.Thread(target=scrape_url, args=(url,data["keywords"]))
#     thread.start()
#     threads.append(thread)
    
# for thread in threads:
#     thread.join()